In [1]:
pip install adjustText

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from adjustText import adjust_text

In [ ]:
df=pd.read_csv('owid-covid-data.csv')

In [ ]:
default_fields=['new_cases', 'new_deaths', 'icu_patients']

def pull(country='', continent='', start=0, cols=default_fields):
    #Pull cols+date data for given country/continent after start days
    
    file='owid-covid-data.csv'
    df=pd.read_csv(file)
    if 'date' not in cols:
        cols=['date']+cols
    if country:
        df=df[df.location == country].loc[:,cols]
    elif continent: 
        df=df[df.continent == continent].loc[:,cols]
    else:
        df=df.loc[:,cols]
    df.date=df.date.apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))
    s=df.date.iloc[0]
    df=df[(df.date-s).dt.days >= start].set_index('date')
    return df

$$\textbf{Q1: Which countries have suffered the worst Covid outbreaks?}$$
In particular we'd like to find out which countries have the highest death/case counts proprtionate to their population. 

In [ ]:
df=pull(cols=['location','iso_code','total_deaths_per_million', 'total_cases_per_million']).loc['2021-02-1']
df=df.dropna(subset=['iso_code']).drop('iso_code', axis=1).set_index('location').drop('World').dropna()# drop continents/the world

In [ ]:
deaths_rank=df[['total_deaths_per_million']].sort_values('total_deaths_per_million', ascending=False).take(range(10)).reset_index()
deaths_rank.index+=1
deaths_rank.columns= ['Country', 'Deaths per million']
deaths_rank


The ten worst outbreaks, as measured by deaths, consist almost entirely of European Countries. The exception being of course the United States coming in at number 10.

In [ ]:
cases_rank=df[['total_cases_per_million']].sort_values('total_cases_per_million', ascending=False).take(range(10)).reset_index()
cases_rank.index+=1
cases_rank.columns= ['Country', 'Cases per million']
cases_rank

The countries with the most cases per million are a little less dominated by European Countries. Somewhat surprisngly only 4 countries appear on both lists. This could be a consequence of understated case counts in countries overwhelmed by the disease, or there might be an underlying epidemeiological cause such variations in population demographics making some countries more vulnarable.

$$\textbf{Q2: Is there a correlation between population density and outbreak severity?}$$
Once again we'll consider both total cases and total deaths as possible measures of outbreak severity. For this question we'll focus mainly on Europe, since this block is relatively similar and heavily affected. Plus there is good quality data on population density.

There are also two main measures of population density we might use. The first is the usual. $$Density=\frac{Population}{Area}$$

Alternatively we can use population weighted density. In principle for a country with population $X$ we have

$$\text{PWD}(d,X) = \frac{1}{|X|}\sum_{p \in X}N(p,d)$$
where $N(p,d)$ is the number of people living within $d$ of $p$. In practice this number is not computable, but we can break a country up into a set of pixels P_{i} of fixed area $a$, density $d_{i}$ and population $p_{i}$. Then 

$$\text{APWD}(a, X) =\frac{\sum_{P_{i}} d_{i}p_{i}}{\sum p_{i}} $$
gives an approximate measure, though it depends on our choice of pixels. We will take the population weighted density to be given by $AWD(1,X)$.

In [ ]:
file='WorldBank Population density.csv'
dense= pd.read_csv(file).set_index('Country Name')

df=pull(continent='Europe', cols=['location','iso_code','total_deaths_per_million', 'total_cases_per_million']).loc['2021-02-1']
df=df.dropna(subset=['iso_code']).drop('iso_code', axis=1).set_index('location').dropna()# drop continents/the world

In [ ]:
df=df.merge(dense['2018'], how= 'inner', left_on='location', right_index=True)
df.rename(columns={'2018':'Population Density'}, inplace=True)
def inp(row,n):
    if n > 10:
        return row
    if row['Population Density']!= row['Population Density']:
        row['Population Density']= dense.loc[row.name, str(2017-n)]
        return inp(row,n+1)
    else:
        return row
df=df.apply(lambda row: inp(row, 0), axis=1).sort_values('Population Density')

In [ ]:
file='pwd_country.csv'
pwd= pd.read_csv(file)
pwd=pwd[['COUNTRY_NAME', '2010']] ## Data for 2020 onwards are just estimates
pwd.columns=['Country Name', 'Population Weighted Density']
df=df.merge(pwd, left_index=True, right_on='Country Name', how='inner').set_index('Country Name')
df.rename(columns={'total_deaths_per_million':'Total Deaths Per Million', 'total_cases_per_million':'Total Cases Per Million'}, inplace=True)

In [ ]:
fig=sns.lmplot(data=df.drop(['Malta']), y='Total Cases Per Million', x='Population Density', fit_reg=True, aspect=3, height=5)
fig.set(xlim=(0, df.drop(['Malta'])['Population Density'].max()+10))

labels=[]

for country in df.drop(['Malta']).index:
     labels.append(plt.text(df.loc[country,'Population Density']+0.01, df.loc[country, 'Total Cases Per Million'], 
     country))
labels
adjust_text(labels, arrowprops=dict(arrowstyle="->", color='black'));
plt.title('Density vs Cases in Europe')

If we plot total cases per million vs population density then there does seem to be something of a correlation. The line of best clearly doesn't give a good description of the relationship, so the Pearson Coefficient will be low. However we would expect a Spearman Rank calculation to find a modest correlation.

In [ ]:
fig=sns.lmplot(data=df, y='Total Deaths Per Million', x='Population Weighted Density', fit_reg=True, aspect=3, height=5)
fig.set(xlim=(df['Population Weighted Density'].min()-50, df['Population Weighted Density'].max()+50))

labels=[]

for country in df.index:
     labels.append(plt.text(df.loc[country,'Population Weighted Density']+0.01, df.loc[country, 'Total Deaths Per Million'], 
     country))
labels
adjust_text(labels, arrowprops=dict(arrowstyle="->", color='black'));
plt.title('Population Weighted Density vs Cases in Europe')

Very surprisingly, to me at least. There appears to be little relationship at all - the line of best fit is basically flat, and it is a poor desciptor besides. 

In [ ]:
res=df.corr(method='pearson').drop(['Population Density', 'Population Weighted Density'], axis=1).merge(
    df.corr(method='spearman').drop(['Population Density', 'Population Weighted Density'], axis=1), left_index=True, right_index=True, suffixes=('-Pearson', '-Spearman')).loc[['Population Density', 'Population Weighted Density']]
res

The correlation calculations backup the graphical observations. Neither measure of density has a strong linear correlation with deaths or cases. On the other hand the Spearman's rank calculations tell us there is some degree of correlation, with the strongest being between Total Cases and Population Density.

$$\textbf{Q3: Is there good linear model which predicts deaths based upon case and/or ICU patient numbers?}

Many commentators have given a variety of rules of thumb for estimate death rates 1/2 weeks in advance. Intuitively we would expect some essentially fixed proportion of ICU patints/ Covid cases not to survive, so some kind of linear model is a good candidate. Indeed all the estimates I have seen given are of this form.

There are of course complicating factors, most noticeably noise introduced to cases and deaths by periodic reporting cycles and that the new cases reported are not always a good estimate of the actual number of cases in the population. 

To answer this question we'll construct a few models on data from 2020 which we'll test on data from January 2021 to give us an idea of real world performance. Obviously it's not ideal that the test data isn't chosen at random, but it will do for our purposes.

In [ ]:
df=pull(country='United Kingdom')
df.index.rename('Date', inplace=True)
df.columns=['New Cases', 'New Deaths', 'ICU Patients']
df.isna().sum()

In [ ]:
fig=plt.figure(figsize=(16, 6))
ax=sns.lineplot(data=df, x=df.index, y='ICU Patients', color='red', alpha=0.5)
sns.lineplot(data=df, x=df.index, y='New Deaths', ax=ax, color='green', alpha=0.5)
ax.set(ylabel='')
plt.legend(['ICU Patients', 'New Deaths']);
ax2=ax.twinx()
ax=sns.lineplot(data=df, x=df.index, y='New Cases', ax=ax2)
plt.legend(['New Cases']);
plt.title('Covid in the UK')

There is some missing data at the beginning of the pandemic. We can fill the NaN's with 0 but for the most part we'll avoid using data before mid-April, even when we have it the quality is very questionable. It can't be seen on the graph but the 2 most recent days of Icu patient numbers are missing too, these rows will have to be dropped.

Looking at the data, it does loosely seem, at least in the second half of the year, that the number of deaths follows ICU patients which in turn follows New Cases. As above however, there is a large amount of noise in both the death and case numbers.

In [ ]:
df=df.fillna(0).iloc[:-2,:]

In [ ]:
def trail(df,cols,trail_from, trail_to=0, keepNan=False):
    #df must be date indexed
    #Trail_from and trail_to are either an integers or lists of length =col, can mix and match 
    #Includes data from between trail_from and trail_to days before
    #keepNan=True includes dates for which the trailing window has missing entries
    
    if not isinstance(trail_from, list) and not isinstance(trail_to, list):  
        odf=pd.DataFrame(df[cols].copy())
        if not keepNan:
            ndf=df.iloc[trail_from:,:]
        else:
            ndf=df
        for i in range(0,trail_from+1,1):
            if i >= trail_to:
                ndf=ndf.merge(odf, how='left', left_index=True, right_index=True, suffixes=('','_-'+str(i)) )
            diff=dt.timedelta(days=1)
            odf.index=odf.index+diff
        
        ndf=ndf.drop(cols,axis=1)  
        ndf=ndf
        return ndf
    else:
        if not isinstance(trail_from, list):
            trail_from =len(cols)*[trail_from]
             
        if not isinstance(trail_to, list):
            trail_to =len(cols)*[trail_to]
        
        if len(cols)!=len(trail_from):
            raise ValueError('Trail_from must be the same length as cols')
        elif len(cols)!=len(trail_to):
            raise ValueError('Trail_to must be the same length as cols')
        else:
            if not keepNan:
                ndf=df.iloc[max(trail_from):,:]
            else:
                ndf=df
            
            for i in range(len(cols)):
                col=cols[i]
                n=trail_from[i]
                m=trail_to[i]
                #print(n,m,col)
                ndf=ndf.drop(col,axis=1).merge(trail(pd.DataFrame(df[col]), [col], n,m, keepNan=keepNan)
                                          ,how='left', left_index=True, right_index=True, suffixes=('x','y'))
            
        return ndf
def pipe(df, target='New Deaths', predictors=['New Cases', 'ICU Patients'],trailing=['New Cases', 'ICU Patients'], trail_n=21,trail_to=7, smooth=7, cutoffs=(dt.date(2020,1,1)
, dt.date(2021, 1, 1))):
    #target (str)- variable to model
    #trailing (str/list of strs)- categories to include historial data for
    #trail_n (int/list of ints) # of days to include takes either single value or one value per category
    #trail_to (int/list of ints) # prevents inclusion of data more recent than trail_to days
    #smooth (int) - if non-zero replace target by n day trailing average
    #cutoffs (pair of datetimes)-restrict attention to dates between those given
    
    df=df[[target]+predictors]        
        
    if smooth:
        target_series=trail(pd.DataFrame(df[target]), target,smooth).mean(axis=1).rename(target+'_smoothed'+str(smooth))
        #print(df.columns)
        df=df.drop(target, axis=1)
        target=target+'_smoothed'+str(smooth)
        #target_Series=target_series.rename(target)
    else:
        target_series=df[target]
        df=df.drop(target, axis=1)
        
    trail_df=trail(df, trailing ,trail_n, trail_to)
    
    if cutoffs:
        target_series=target_series.loc[cutoffs[0]:cutoffs[1]]
    
    if trail_df.shape[0] != target_series.shape[0]:
        merge_dat=trail_df.merge(target_series, how ='inner', left_index=True, right_index=True, suffixes=('',''))
        y=merge_dat.loc[:,target]
        X=merge_dat.drop(target, axis=1)
    else:
        y=target_series
        X=trail_df.drop(target)
    
    
    
    lm_model=LinearRegression()
    lm_model.fit(X,y)
    y_pred=lm_model.predict(X)
    
    return trail_df, X,y, y_pred, lm_model

def plotpred(x,y, y_pred):
    plt.figure(figsize=(16, 6))
    ax=sns.lineplot(x=x, y=y_pred)
    sns.lineplot(x=x, y=y, ax=ax, color='red', alpha=0.5)
    plt.legend(['Prediction','Actual'])
    
def coefdf(model, cols):

    coefs=pd.DataFrame(model.coef_,cols).reset_index()
    coefs.columns=['Variable', 'Coefficient']
    def rowlabel(row):
        label=row[0].split('_')
        try: 
            c=int(label[-1])
            if len(label)==3:
                x=label[0]+'_'+label[1]
            else:
                x=label[0]
        except:
            c=0
            if len(label)==2:
                x=label[0]+'_'+label[1]
            else:
                x=label[0]
        return x,c, row[1]
    coefs=coefs.apply(rowlabel, axis=1, result_type='expand')
    coefs.columns=['DataType', 'Days', 'Coefficient']
    return coefs
    

In [ ]:
trail_df,X,y, y_pred, lm_model=pipe(df, predictors=['New Cases'], trailing=['New Cases'], trail_n=28, trail_to=6, smooth=0,cutoffs=(dt.date(2020,6,1)
, dt.date(2021, 1, 1)))
plotpred(y.index, y, y_pred)
print('MSE-TRAIN: '+str(mean_squared_error(y_pred,y)))
plt.title('Model Estimate - Train')

We can get a pretty good model by doing linear regression on case data from 1-4 weeks prior to the prediction date. The relatively large number of input variables mean we're in danger of overfitting, however, and unsuprsingly the model performs poorly on the test data as seen below.

In [ ]:
_,X,y, _, _=pipe(df, cutoffs=(dt.date(2021,1, 1)
, dt.date(2021, 2, 1)),predictors=['New Cases'], trailing=['New Cases'], trail_to=6,trail_n=28, smooth=0)
y_pred=lm_model.predict(X)
plotpred(y.index, y, y_pred)
print('MSE-TEST: '+str(mean_squared_error(y_pred,y)))
plt.title('Model Estimate - Test')

We could of course try including fewer days of data to combat this, but the performance remains poor. Regardless of the number of days included all the models systematically underestimate the number of deaths in the last few months of the year, it is possible this is caused by a fundamental shift in the relationship between case numbes and deaths, due to say the new variant or increasing pressure on the NHS reaching a tipping point.

The solution would seem to be to include data on ICU patients as well. Since this data is not subject to the same reporting patterns as deaths and cases it is important to smooth out the target data. We will do this by replacing New Deaths with its 7 day trailing average.

In [ ]:
n=21
m=6
s=6

trail_df,X,y, y_pred, lm_model=pipe(df, predictors=['ICU Patients'], trailing=['ICU Patients'], trail_n=n, trail_to=m, smooth=s, cutoffs=(dt.date(2020,4,15)
, dt.date(2021, 1, 1)))
plotpred(y.index, y, y_pred)
plt.title('Model Estimate - Train')
print('MSE-TRAIN: '+str(mean_squared_error(y_pred,y)))
_,X,y, _, _=pipe(df, cutoffs=(dt.date(2021,1, 1)
, dt.date(2021, 2, 1)),predictors=['ICU Patients'], trailing=['ICU Patients'], trail_n=n,trail_to=m, smooth=s)
y_pred=lm_model.predict(X)
plotpred(y.index, y, y_pred)
plt.title('Model Estimate - Test')
print('MSE-TEST: '+str(mean_squared_error(y_pred,y)))

This model performs fantastically, though as before there is some underestimation on the training data in the last couple of months of 2020.

We see however that this model largely predicts from the most recent sets of ICU figures. This is the pattern no matter the date range - the best estimate of deaths on a given day is essentially given by whatever the most recent ICU data is. In particular, as the table beloew shows, the model performs quite poorly if we wish to predict more than about 10 days so in advance.

There is also a large negative contribution from ICU patients 2 weeks before the prediction date. I'm not sure what, if any, conclusions can be drawn from this.

In [ ]:
coefdf(lm_model,X.columns).drop('DataType',axis=1).plot(x='Days', y='Coefficient')
plt.title('Model Coefficients')
plt.show()

def train_model(days):

    trail_df,X,y, y_pred, lm_model=pipe(df, predictors=['ICU Patients'], trailing=['ICU Patients'], trail_n=days+15, trail_to=days, smooth=s, cutoffs=(dt.date(2020,4,10+days)
                                        , dt.date(2021, 1, 1)))
    tr=mean_squared_error(y_pred,y)

    _,X,y, _, _=pipe(df, cutoffs=(dt.date(2021,1, 1)
                    , dt.date(2021, 2, 1)),predictors=['ICU Patients'], trailing=['ICU Patients'], trail_n=days+15, trail_to=days, smooth=s)
    y_pred=lm_model.predict(X)
    te=mean_squared_error(y_pred,y)
    
    return pd.Series([tr,te])

res=pd.Series(range(6,15))
res.index=range(6,15)
res=res.apply(train_model)
res.columns=['MSE-Train', 'MSE-Test']
res.index.rename('Days in Advance', inplace=True)
res

We can construct a model that uses both these features to give pretty good performance. By using both case numbers and ICU patient numbers we can predict smoothed deaths reasonably well using only data from 2-3 weeks prior to the prediction date. It is possible that the falling real death count at the end of January, relative to the predicted rate, is a consequence of the vaccines being distributed but it's too early to tell at this stage.

In [ ]:
s=6
m=[13,13]
n=[17, 17]

trail_df,X,y, y_pred, lm_model=pipe(df, cutoffs=(dt.date(2020,4,15)
, dt.date(2021, 1, 1)), trail_to=m,trail_n=n, smooth=s)
print('MSE-TRAIN: '+str(mean_squared_error(y_pred,y)))
plotpred(y.index, y, y_pred)
plt.title('Model Estimate - Train')
_,X,y, _, _=pipe(df, cutoffs=(dt.date(2021,1, 1)
, dt.date(2021, 2, 1)), trail_to=m,trail_n=n, smooth=s)
y_pred=lm_model.predict(X)
plotpred(y.index, y, y_pred)
plt.title('Model Estimate - Test')
print('MSE-TEST: '+str(mean_squared_error(y_pred,y)))

If you're looking for something simpler in your life and want to predict the deaths a couple of weeks in advance, the following model does a suprisngly good job just using cases and icu_patients 2 weeks prior. Curiously this model outperforms the larger one on the test data, this is concievably a consequence of overfitting in the earlier model but is most likely chance.

In [ ]:
m=[13,13]
n=[13,13]
s=6

trail_df,X,y, y_pred, lm_model=pipe(df, cutoffs=(dt.date(2020,5,1)
, dt.date(2021, 1, 1)), trail_to=m,trail_n=n, smooth=0) ## Not modelled against smoothed data because not enough information provided 
_,X,y, _, _=pipe(df, cutoffs=(dt.date(2020,5, 1)
, dt.date(2021, 1, 1)), trail_to=m,trail_n=n, smooth=s)
print('MSE-TRAIN: '+str(mean_squared_error(y_pred,y)))
plotpred(y.index, y, y_pred)
plt.title('Model Estimate - Train')
_,X,y, _, _=pipe(df, cutoffs=(dt.date(2021,1, 1)
, dt.date(2021, 2, 1)), trail_to=m,trail_n=n, smooth=s)
y_pred=lm_model.predict(X)
plotpred(y.index, y, y_pred)
plt.title('Model Estimate - Test')
print('MSE-TEST: '+str(mean_squared_error(y_pred,y)))

In [ ]:
coefdf(lm_model,X.columns)

Overall we see that cases are consistently poor indicator of future deaths, 

Overall, neither ICU patients nor cases are particularly good predictors of reported deaths at 2 weeks; though, after smoothing the data, the number of ICU patients can be used to predict deaths in the next week extremely well. Combined they offer a pretty good estimation at 2 weeks, even if you restrict to just a single days worth of data.

We could likely estimate deaths even better if we included a contribution of some kind from the positivity rate to estimate how many cases go unnoticed, it's not clear however exactly how this would be best implented, especially if we wished to ensure that relationship would be well approximated by some linear function. You're welcome to give it a try yourself though, if you have a couple of hours free.

There's also this period at the end of 2020 where almost all the models underestimate the number of deaths. Why do you think this is?